In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from io import StringIO
import matplotlib.dates as dates
from datetime import datetime
import matplotlib.ticker as ticker
from random import shuffle
from time import time
from pandas.plotting import register_matplotlib_converters
%matplotlib notebook

#os.chdir('C:\\Users\\A Sua\\Documents\\FIU\\CAP2020')
os.chdir('C:\\Users\\Daniel\\Python and R Notebooks\\CAP2020\\CAP2020')

## Import Data

In [2]:
path = 'data\\processed\\compiled_quarterly_final_v2.csv'
df_quarter = pd.read_csv(path)
df_quarter.index = df_quarter['Quarter end']
df_quarter = df_quarter.drop('Quarter end', axis=1)
df_quarter.index.names = ['qend']


#df_bidlo.head()
#### TRANSPOSE DATA FOR EASE OF USE
# df_quarter = df_quarter.T
df_quarter.index = pd.to_datetime(df_quarter.index)
# print(df_quarter.shape)
df_quarter.head()

,Shares split adjusted,Price,EPS basic,EPS diluted,Dividend per share,Dividend payout ratio,P/E ratio,Long-term debt to equity ratio,Symbol,Quarter
qend,,,,,,,,,,
2019-09-30,67151778,63.99,0.59,0.58,0.035,0.0488,21.69,0.1859,AAN,2019Q3
2019-06-30,67547497,57.23,0.63,0.62,0.035,0.0464,19.94,0.1880,AAN,2019Q2
2019-03-31,67677449,48.63,0.83,0.82,0.035,0.0459,17.49,0.2251,AAN,2019Q1
2018-12-31,67202919,47.00,0.90,0.89,0.040,0.0455,10.83,0.1937,AAN,2018Q4
2018-09-30,68594784,48.86,0.64,0.62,0.030,0.0267,12.00,0.1686,AAN,2018Q3


## Change Columns

In [3]:
df_quarter.columns = ['sharesout', 'price', 'eps_basic', 'eps_diluted', 'div_share', 'div_payout', 'p/e', 'debt/eq', 'symbol', 'quarter']
df_quarter

,sharesout,price,eps_basic,eps_diluted,div_share,div_payout,p/e,debt/eq,symbol,quarter
qend,,,,,,,,,,
2019-09-30,67151778,63.99,0.59,0.58,0.035,0.0488,21.69,0.1859,AAN,2019Q3
2019-06-30,67547497,57.23,0.63,0.62,0.035,0.0464,19.94,0.1880,AAN,2019Q2
2019-03-31,67677449,48.63,0.83,0.82,0.035,0.0459,17.49,0.2251,AAN,2019Q1
2018-12-31,67202919,47.00,0.90,0.89,0.040,0.0455,10.83,0.1937,AAN,2018Q4
2018-09-30,68594784,48.86,0.64,0.62,0.030,0.0267,12.00,0.1686,AAN,2018Q3
...,...,...,...,...,...,...,...,...,...,...
2014-03-30,501018690,30.75,0.31,0.31,0.072,0.2623,30.45,3.3816,ZTS,2014Q1
2013-12-31,500729429,32.10,0.21,0.21,0.070,0.2630,NaN,3.8745,ZTS,2013Q4
2013-09-29,500006833,30.78,0.26,0.26,0.065,NaN,NaN,4.3203,ZTS,2013Q3


## Change data types

In [4]:
df_quarter['symbol'] = df_quarter['symbol'].astype(str)
df_quarter['quarter'] = df_quarter['quarter'].astype(str)
df_quarter['sharesout'] = df_quarter['sharesout'].astype(int)
df_quarter['price'] = df_quarter['price'].astype(float)
df_quarter['eps_basic'] = df_quarter['eps_basic'].astype(float)
df_quarter['eps_diluted'] = df_quarter['eps_diluted'].astype(float)
df_quarter['div_share'] = df_quarter['div_share'].astype(float)
df_quarter['div_payout'] = df_quarter['div_payout'].astype(float)
df_quarter['p/e'] = df_quarter['p/e'].astype(float)
df_quarter['debt/eq'] = df_quarter['debt/eq'].astype(float)

In [5]:
# fix some sharesout values being negative
df_quarter['sharesout'] = df_quarter['sharesout'].abs()

## Link up Quarter to Quarter end

In [6]:
df_quarter['new_key'] = df_quarter['symbol'] + '_'+ df_quarter['quarter']

In [7]:
dates_table = pd.read_csv('data\\external\\quarter_symbols.csv')
dates_table['std_date']  = pd.to_datetime(dates_table['std_date'])
dates_table['index'] = dates_table['Symbol'] + '_' + dates_table['Quarter']

In [8]:
df_merge = pd.merge(dates_table, df_quarter, left_on='index', right_on='new_key', how='inner')

In [9]:
symbol_list = df_merge['symbol'].unique()

## Market Cap

In [10]:
df_merge['market_cap'] = df_merge['sharesout'] * df_merge['price']

In [11]:
df_reduced = df_merge[['std_date', 'symbol', 'market_cap']]
df_reduced

,std_date,symbol,market_cap
0,2018-12-31,AAN,3.158537e+09
1,2018-09-30,AAN,3.351541e+09
2,2018-06-30,AAN,3.034743e+09
3,2018-03-31,AAN,3.023421e+09
4,2017-12-31,AAN,2.779977e+09
...,...,...,...
32001,2014-03-31,ZTS,1.540632e+10
32002,2013-12-31,ZTS,1.607341e+10
32003,2013-09-30,ZTS,1.539021e+10
32004,2013-06-30,ZTS,1.603500e+10


In [12]:
########----------------#######
df_newq = pd.DataFrame([])

for symbol in symbol_list:
    df_temp = df_reduced[df_reduced['symbol']==symbol]
    df_temp.index = df_temp['std_date']
    df_temp = df_temp[['market_cap']]
    df_temp = df_temp.T
    df_temp.index = [symbol]
    df_newq = pd.concat([df_newq, df_temp], axis=0)
    #df_temp.head()

df_newq.head()

std_date,2006-12-31,2007-03-31,2007-06-30,2007-09-30,2007-12-31,2008-03-31,2008-06-30,2008-09-30,2008-12-31,2009-03-31,...,2016-12-31,2017-03-31,2017-06-30,2017-09-30,2017-12-31,2018-03-31,2018-06-30,2018-09-30,2018-12-31,2019-03-31
AAN,NaN,1.530424e+09,1.529722e+09,1.352419e+09,1.100907e+09,9.681865e+08,1.248318e+09,1.378276e+09,1.180735e+09,1.323695e+09,...,2.023723e+09,2.084473e+09,2.453513e+09,3.020149e+09,2.779977e+09,3.023421e+09,3.034743e+09,3.351541e+09,3.158537e+09,NaN
AAPL,NaN,2.259415e+10,2.781990e+10,3.493417e+10,4.689335e+10,4.233140e+10,4.557635e+10,4.152004e+10,2.755189e+10,2.620533e+10,...,1.059862e+11,1.191135e+11,1.291119e+11,1.289905e+11,1.284101e+11,1.034879e+11,9.490702e+10,9.303196e+10,7.987202e+10,NaN
AAP,NaN,4.029422e+09,4.234284e+09,3.488207e+09,3.414902e+09,3.295897e+09,3.602546e+09,3.852041e+09,2.923150e+09,3.530591e+09,...,1.150386e+10,1.172801e+10,9.263846e+09,7.301866e+09,6.902159e+09,8.277441e+09,9.095697e+09,1.129853e+10,1.198755e+10,NaN
AA,NaN,2.788560e+10,3.382606e+10,3.351030e+10,3.009912e+10,2.704441e+10,3.189319e+10,2.264629e+10,1.168452e+10,8.482691e+09,...,1.076229e+10,1.087078e+10,1.111398e+10,1.191759e+10,1.221414e+10,1.297370e+10,9.789492e+09,9.606858e+09,9.514539e+09,NaN
ABBV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.571491e+10,1.003148e+11,1.090201e+11,1.279698e+11,1.463634e+11,1.721606e+11,1.479899e+11,1.406442e+11,1.284060e+11,NaN


In [13]:
df_newq = df_newq.iloc[:, 1:-1]
df_newq.shape

(761, 48)

In [14]:
new_symbols_div = []
null_symbols_div = []
for symbol in df_newq.index:
    temp_value = df_newq.loc[symbol, :].isnull().sum()
    if temp_value == 0:
        new_symbols_div.append(symbol)
    else:
        null_symbols_div.append(symbol)

In [15]:
df_quarter_div = pd.DataFrame([])
for symbol in new_symbols_div:
    temp_df = df_newq[df_newq.index == symbol]
    df_quarter_div = pd.concat([df_quarter_div, temp_df])

df_quarter_div.shape

(515, 48)

In [16]:
export_path = 'data\\processed\\df_quarter_market_cap_07-18.csv'
df_quarter_div.to_csv(export_path)

In [17]:
df_quarter_div.head()

std_date,2007-03-31,2007-06-30,2007-09-30,2007-12-31,2008-03-31,2008-06-30,2008-09-30,2008-12-31,2009-03-31,2009-06-30,...,2016-09-30,2016-12-31,2017-03-31,2017-06-30,2017-09-30,2017-12-31,2018-03-31,2018-06-30,2018-09-30,2018-12-31
AAPL,2.259415e+10,2.781990e+10,3.493417e+10,4.689335e+10,4.233140e+10,4.557635e+10,4.152004e+10,2.755189e+10,2.620533e+10,3.514859e+10,...,1.077180e+11,1.059862e+11,1.191135e+11,1.291119e+11,1.289905e+11,1.284101e+11,1.034879e+11,9.490702e+10,9.303196e+10,7.987202e+10
AAP,4.029422e+09,4.234284e+09,3.488207e+09,3.414902e+09,3.295897e+09,3.602546e+09,3.852041e+09,2.923150e+09,3.530591e+09,4.109087e+09,...,1.171166e+10,1.150386e+10,1.172801e+10,9.263846e+09,7.301866e+09,6.902159e+09,8.277441e+09,9.095697e+09,1.129853e+10,1.198755e+10
AA,2.788560e+10,3.382606e+10,3.351030e+10,3.009912e+10,2.704441e+10,3.189319e+10,2.264629e+10,1.168452e+10,8.482691e+09,9.457899e+09,...,1.315435e+10,1.076229e+10,1.087078e+10,1.111398e+10,1.191759e+10,1.221414e+10,1.297370e+10,9.789492e+09,9.606858e+09,9.514539e+09
ABC,9.366824e+09,9.242960e+09,7.749728e+09,7.314020e+09,6.987997e+09,6.449381e+09,6.311239e+09,4.909430e+09,5.055251e+09,5.228754e+09,...,1.871512e+10,1.633509e+10,1.897078e+10,1.942643e+10,1.889245e+10,1.826988e+10,2.078687e+10,1.910213e+10,1.799315e+10,1.733048e+10
ABMD,4.414972e+08,3.971059e+08,3.945132e+08,4.508959e+08,4.617211e+08,5.541965e+08,6.652908e+08,5.242521e+08,3.996504e+08,2.554858e+08,...,5.204418e+09,4.965226e+09,5.029583e+09,5.839568e+09,6.865883e+09,8.082474e+09,1.094866e+10,1.627156e+10,1.800455e+10,1.654108e+10


In [18]:
df_quarter_div.loc['AAPL']

std_date
2007-03-31    2.259415e+10
2007-06-30    2.781990e+10
2007-09-30    3.493417e+10
2007-12-31    4.689335e+10
2008-03-31    4.233140e+10
2008-06-30    4.557635e+10
2008-09-30    4.152004e+10
2008-12-31    2.755189e+10
2009-03-31    2.620533e+10
2009-06-30    3.514859e+10
2009-09-30    4.640587e+10
2009-12-31    5.718529e+10
2010-03-31    6.256998e+10
2010-06-30    7.173507e+10
2010-09-30    8.034843e+10
2010-12-31    9.186982e+10
2011-03-31    1.037583e+11
2011-06-30    9.986919e+10
2011-09-30    1.116646e+11
2011-12-31    1.150934e+11
2012-03-31    1.504957e+11
2012-06-30    1.689401e+11
2012-09-30    1.826237e+11
2012-12-31    1.696707e+11
2013-03-31    1.405093e+11
2013-06-30    1.254810e+11
2013-09-30    1.309291e+11
2013-12-31    1.461132e+11
2014-03-31    1.306923e+11
2014-06-30    1.422882e+11
2014-09-30    1.537062e+11
2014-12-31    1.644055e+11
2015-03-31    1.746227e+11
2015-06-30    1.813470e+11
2015-09-30    1.440153e+11
2015-12-31    1.433184e+11
2016-03-31    1.193

In [19]:
df_quarter[df_quarter['symbol']=='AAPL']

,sharesout,price,eps_basic,eps_diluted,div_share,div_payout,p/e,debt/eq,symbol,quarter,new_key
qend,,,,,,,,,,,
2019-09-28,148297704,209.50,3.07,3.11,0.77,0.2480,17.86,1.0146,AAPL,2019Q3,AAPL_2019Q3
2019-06-29,224212704,192.79,2.20,2.18,0.77,0.2468,16.21,0.8806,AAPL,2019Q2,AAPL_2019Q2
2019-03-30,306107704,169.84,2.47,2.46,0.73,0.2412,13.97,0.8521,AAPL,2019Q1,AAPL_2019Q1
2018-12-29,420312704,190.03,4.22,4.18,0.73,0.2276,16.01,0.7888,AAPL,2018Q4,AAPL_2018Q4
2018-09-29,450430704,206.54,2.94,2.91,0.73,0.2231,18.74,0.8748,AAPL,2018Q3,AAPL_2018Q3
...,...,...,...,...,...,...,...,...,...,...,...
1995-03-31,902529352,1.46,0.59,NaN,0.12,NaN,NaN,0.1120,AAPL,1995Q1,AAPL_1995Q1
1994-12-30,913630652,1.36,1.55,NaN,0.12,NaN,NaN,0.1187,AAPL,1994Q4,AAPL_1994Q4
1994-09-30,938007592,1.12,NaN,NaN,0.00,NaN,NaN,0.1278,AAPL,1994Q3,AAPL_1994Q3
